In [1]:
%reload_ext autoreload
%autoreload 2

import os
from pathlib import Path
import multiprocessing

import jax
import jax.numpy as jnp
import pandas as pd
import numpy as np
import numpyro
import numpyro.distributions as dist

from hbmep.config import Config
from hbmep.model import Baseline
from hbmep.model.utils import Site as site
from hbmep.utils.constants import RECTIFIED_LOGISTIC

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()
numpyro.enable_validation()


In [2]:
root_path = Path(os.getcwd()).parent.parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/rcml/adm.toml")

config = Config(toml_path=toml_path)


2023-08-09 13:12:21,589 - hbmep.config - INFO - Verifying configuration ...
2023-08-09 13:12:21,589 - hbmep.config - INFO - Success!


#### Load data and preprocess

In [3]:
class RectifiedLogistic(Baseline):
    LINK = RECTIFIED_LOGISTIC

    def __init__(self, config: Config):
        super(RectifiedLogistic, self).__init__(config=config)

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0, ...].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_subject, n_subject, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(
                    site.mu_a,
                    dist.TruncatedNormal(150, 50, low=0)
                )
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(50))

                sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(0.1))

                sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(0.05))
                sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(5))
                sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(10))

                with numpyro.plate("n_feature0", n_feature0, dim=-3):
                    """ Priors """
                    a = numpyro.sample(
                        site.a,
                        dist.TruncatedNormal(mu_a, sigma_a, low=0)
                    )
                    b = numpyro.sample(site.b, dist.HalfNormal(sigma_b))

                    L = numpyro.sample(site.L, dist.HalfNormal(sigma_L))
                    H = numpyro.sample(site.H, dist.HalfNormal(sigma_H))
                    v = numpyro.sample(site.v, dist.HalfNormal(sigma_v))

                    g_1 = numpyro.sample(site.g_1, dist.Exponential(0.01))
                    g_2 = numpyro.sample(site.g_2, dist.Exponential(0.01))

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[feature0, subject]
            + jnp.maximum(
                0,
                -1
                + (H[feature0, subject] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[feature0, subject], v[feature0, subject]) - 1)
                    * jnp.exp(-b[feature0, subject] * (intensity - a[feature0, subject])),
                    1 / v[feature0, subject]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[feature0, subject] + g_2[feature0, subject] * (1 / mu)
        )

        """ Observation """
        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


model = RectifiedLogistic(config=config)


2023-08-09 13:12:21,622 - hbmep.model.baseline - INFO - Initialized model with rectified_logistic link


In [4]:
df = pd.read_csv(model.csv_path)

ind = df.pulse_amplitude.isin([0])
df = df[~ind].reset_index(drop=True).copy()

subset = ['-C5L', '-C6L', '-C7L', '-C8L']
ind = df.compound_position.isin(subset)
df = df[ind].reset_index(drop=True).copy()

# df[model.features] = \
#     df.compound_position \
#         .apply(lambda x: x.split("-")) \
#         .apply(lambda x: (x[0][:2] + "-" + x[1][:2], x[0][-1])) \
#         .apply(pd.Series)

""" Process """
df, encoder_dict = model.load(df=df)

""" Plot """
model.plot(df=df, encoder_dict=encoder_dict)

2023-08-09 13:12:21,648 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/J_RCML_000/adm
2023-08-09 13:12:21,649 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/adm
2023-08-09 13:12:21,650 - hbmep.dataset.core - INFO - Processing data ...
2023-08-09 13:12:21,652 - hbmep.utils.utils - INFO - func:load took: 0.00 sec
2023-08-09 13:12:21,654 - hbmep.dataset.core - INFO - Plotting dataset ...


2023-08-09 13:12:28,848 - hbmep.dataset.core - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/adm/dataset.pdf
2023-08-09 13:12:28,848 - hbmep.utils.utils - INFO - func:plot took: 7.20 sec


In [5]:
df.shape

(1604, 44)

In [6]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-08-09 13:12:28,885 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-08-09 13:59:07,324 - hbmep.utils.utils - INFO - func:run_inference took: 46 min and 38.44 sec


In [7]:
mcmc.print_summary(prob=.95)



                mean       std    median      2.5%     97.5%     n_eff     r_hat
  H[0,0,0]      0.89      0.97      0.73      0.57      1.29    292.58      1.01
  H[0,1,0]      2.43      3.68      0.90      0.04      9.77   2081.26      1.00
  H[0,2,0]      2.42      0.09      2.42      2.26      2.61   2445.42      1.00
  H[0,3,0]      3.86      3.81      2.77      0.16     10.92   2523.29      1.00
  H[0,4,0]      2.11      2.41      1.26      0.24      6.74   3518.11      1.00
  H[0,5,0]      0.09      0.01      0.09      0.07      0.11   4191.88      1.00
  H[0,6,0]      1.83      2.51      0.75      0.20      6.66   2637.48      1.00
  H[0,7,0]      1.70      2.14      1.03      0.00      5.85   2348.20      1.01
  H[1,0,0]      2.40      2.70      1.42      0.43      7.79   1399.37      1.00
  H[1,1,0]      3.31      3.75      1.97      0.18     10.66   2358.96      1.00
  H[1,2,0]      2.49      0.06      2.49      2.37      2.61   3243.58      1.00
  H[1,3,0]      3.60      3

In [8]:
model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)



2023-08-09 13:59:08,167 - hbmep.model.baseline - INFO - Generating predictions ...


2023-08-09 13:59:36,285 - hbmep.utils.utils - INFO - func:predict took: 28.11 sec
2023-08-09 13:59:36,391 - hbmep.model.baseline - INFO - Rendering recruitment curves ...
2023-08-09 13:59:46,029 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/adm/recruitment_curves.pdf
2023-08-09 13:59:46,029 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 37.86 sec


In [9]:
model.render_predictive_check(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-08-09 13:59:46,063 - hbmep.model.baseline - INFO - Generating predictions ...
2023-08-09 14:00:14,038 - hbmep.utils.utils - INFO - func:predict took: 27.97 sec
2023-08-09 14:00:14,223 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...
2023-08-09 14:00:32,892 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/adm/posterior_predictive_check.pdf
2023-08-09 14:00:32,895 - hbmep.utils.utils - INFO - func:render_predictive_check took: 46.83 sec


In [10]:
model.save(mcmc=mcmc)

2023-08-09 14:00:32,963 - hbmep.model.baseline - INFO - Saving inference data ...
2023-08-09 14:00:51,562 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/adm/mcmc.nc
2023-08-09 14:00:51,563 - hbmep.model.baseline - INFO - Rendering convergence diagnostics ...
2023-08-09 14:01:23,132 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/adm/diagnostics.csv
2023-08-09 14:01:23,133 - hbmep.model.baseline - INFO - Evaluating model ...
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1037: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is 